In [15]:
import numpy as np
import h5py
import scipy.io
from sklearn import metrics
import pandas as pd
import os
os.environ['THEANO_FLAGS'] = "device=cuda0,force_device=True,floatX=float32,gpuarray.preallocate=0.3"
import theano
print(theano.config.device)
from keras.layers import Embedding
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Layer, merge, Input, Concatenate, Reshape, concatenate,Lambda,multiply,Permute,Reshape,RepeatVector
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras import backend as K

cuda0


## Download data

Download the ChIP-seq data used in this work in <http://deepsea.princeton.edu/media/code/deepsea_train_bundle.v0.9.tar.gz>. 

Ater unzipping the 'deepsea_train_bundle.v0.9.tar.gz' file, locate the 'train.mat', 'valid.mat' and 'test.mat' files in the 'deepsea_train' folder into the './data' folder

## Load data (training and validation)

In [ ]:
data_folder = "./data/"

trainmat = h5py.File(data_folder+'train.mat')
validmat = scipy.io.loadmat(data_folder+'valid.mat')

X_train = np.transpose(np.array(trainmat['trainxdata']),axes=(2,0,1))
y_train = np.array(trainmat['traindata']).T

trainmat.close()

loading data


## Choose only the targets that correspond to the TF binding

In [4]:
y_train = y_train[:,125:815]

## Run CLARINET
CLARINET is built upon keras implementation of DanQ*.

*Quang, Daniel, and Xiaohui Xie. "DanQ: a hybrid convolutional and recurrent deep neural network for quantifying the function of DNA sequences." Nucleic acids research 44.11 (2016): e107-e107.

In [ ]:
kernel_sizes = [8,12,16,20,24]
kernel_nums = [64,64,64,64,64]
kernel_parms = list(zip(kernel_sizes,kernel_nums))

sequence_input = Input(shape=(1000,4))

#CNN layer
conv_outputs =[]
for kernel_size, kernel_num in kernel_parms:
    conv_output = Conv1D(kernel_num,kernel_size=kernel_size,padding="valid",activation="relu")(sequence_input)


    conv_output = MaxPooling1D(pool_size=13, strides=13)(conv_output)
    conv_output = Lambda(lambda x : x[:,:75,:],output_shape=(75, 64))(conv_output)
    conv_outputs.append(conv_output)
output = Concatenate()(conv_outputs)
output = Dropout(0.2)(output)

#BiLSTM layer
output = Bidirectional(LSTM(320,return_sequences=True))(output)
output = Dropout(0.5)(output)


# Attention layer
p = 100
attention = Dense(p)(output)
attention = Permute((2, 1))(attention)
attention = Activation('softmax')(attention)
attention = Permute((2, 1))(attention)
attention = Lambda(lambda x: K.mean(x, axis=2), name='attention',output_shape=(75,))(attention)
attention = RepeatVector(640)(attention)
attention = Permute((2,1))(attention)

output = multiply([output, attention])

#Fully connected layer
output = Flatten()(output)

output = Dense(695)(output)
output = Activation('relu')(output)

#Output layer
output = Dense(690)(output)
output = Activation('sigmoid')(output)

model = Model(inputs=sequence_input, outputs=output)

print('compiling model')
model.compile(loss='binary_crossentropy', optimizer='adam')

print('running at most 60 epochs')

print('model summary')
model.summary()

checkpointer = ModelCheckpoint(filepath="./models/CLARINET_.{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1, save_best_only=False)
earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

model.fit(X_train, y_train, batch_size=100, epochs=60, shuffle=True, verbose=1, validation_data=(np.transpose(validmat['validxdata'],axes=(0,2,1)),validmat['validdata'][:,125:815]), callbacks=[checkpointer,earlystopper])


model.save('./models/CLARINET.h5')

compiling model
running at most 60 epochs
model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1000, 4)      0                                            
__________________________________________________________________________________________________
conv1d_21 (Conv1D)              (None, 993, 64)      2112        input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 989, 64)      3136        input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, 985, 64)      4160        input_5[0][0]                    
_____________________________________________________